<a href="https://colab.research.google.com/github/ormorteey/STA-232-Prequals/blob/main/STA_232_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
rm(list = ls())

In [ ]:
install.packages("pacman")
library(pacman)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
p_load(tidyverse, lme4, googledrive)

In [ ]:
drive_deauth()
drive_user()
public_file = drive_get(as_id("1fm15wxA4Hl2xNsRw6jKE3dQbqqYoOB7f"))
le_inf0 = 1:nrow(public_file) %>% map(~drive_download(public_file[.,], overwrite = TRUE))

ℹ Not logged in as any specific Google user.

File downloaded:

• long_dat.csv <id: 1fm15wxA4Hl2xNsRw6jKE3dQbqqYoOB7f>

Saved locally as:

• long_dat.csv



In [ ]:
election_df = read.csv("long_dat.csv", header = T)
data_df = election_df %>% select(c("cycle", "state", "candidate", "logp"))
head(data_df)

,cycle,state,candidate,logp
,<int>,<chr>,<int>,<dbl>
1,2016,Alabama,1,-0.53088991
2,2016,Alabama,1,0.10110730
3,2016,Alabama,1,0.04546237
4,2016,Alaska,1,0.25009936
5,2016,Alaska,1,-0.45460369
6,2016,Alaska,1,0.11352383


In [ ]:
data_df$candidate = factor(data_df$candidate)
levels(data_df$candidate) = c("0","1")
data_df$cycle = factor(data_df$cycle)
levels(data_df$cycle) = c("0","1")
data_df$state = factor(data_df$state)


In [ ]:
data_df$ran_var = 1:nrow(data_df)
for(i in 1:nrow(data_df)){
data_df$ran_var[i] = paste0(data_df$state[i],data_df$candidate[i] )
}
head(data_df)

,cycle,state,candidate,logp,ran_var
,<fct>,<fct>,<fct>,<dbl>,<chr>
1,0,Alabama,0,-0.53088991,Alabama0
2,0,Alabama,0,0.10110730,Alabama0
3,0,Alabama,0,0.04546237,Alabama0
4,0,Alaska,0,0.25009936,Alaska0
5,0,Alaska,0,-0.45460369,Alaska0
6,0,Alaska,0,0.11352383,Alaska0


In [ ]:
model_1 = lmer(logp ~ candidate + cycle + candidate:cycle + (1 | ran_var), data_df)
summary(model_1)

Linear mixed model fit by REML ['lmerMod']
Formula: logp ~ candidate + cycle + candidate:cycle + (1 | ran_var)
   Data: data_df

REML criterion at convergence: -3307.6

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-6.6992 -0.4271  0.0986  0.5596  7.8295 

Random effects:
 Groups   Name        Variance Std.Dev.
 ran_var  (Intercept) 0.004932 0.07023 
 Residual             0.006716 0.08195 
Number of obs: 1648, groups:  ran_var, 102

Fixed effects:
                   Estimate Std. Error t value
(Intercept)       -0.054452   0.010840  -5.023
candidate1        -0.082099   0.015330  -5.356
cycle1             0.071667   0.005975  11.995
candidate1:cycle1 -0.018251   0.008450  -2.160

Correlation of Fixed Effects:
            (Intr) cnddt1 cycle1
candidate1  -0.707              
cycle1      -0.243  0.172       
cnddt1:cyc1  0.172 -0.243 -0.707

In [ ]:
EBLUP = predict(model_1)
head(EBLUP)

1           2           3           4           5           6 
-0.06248920 -0.06248920 -0.06248920 -0.06007228 -0.06007228 -0.06007228

In [ ]:
head(as.data.frame(ranef(model_1)))

,grpvar,term,grp,condval,condsd
,<chr>,<fct>,<fct>,<dbl>,<dbl>
1,ran_var,(Intercept),Alabama0,-0.008036842,0.02678396
2,ran_var,(Intercept),Alabama1,0.011288350,0.02678396
3,ran_var,(Intercept),Alaska0,-0.005619922,0.02545874
4,ran_var,(Intercept),Alaska1,-0.001219731,0.02545874
5,ran_var,(Intercept),Arizona0,-0.018151705,0.01259119
6,ran_var,(Intercept),Arizona1,0.020251993,0.01259119


In [ ]:
mySumm <- function(mod) {
   c(sigma_e = sigma(mod), sigma_r = sqrt(unlist(VarCorr(mod))))
}
booted_lamb_fitted <- bootMer(model_1, mySumm, nsim = 100, seed = 2047)

In [ ]:
library(boot) # for nice print-out
booted_lamb_fitted


PARAMETRIC BOOTSTRAP


Call:
bootMer(x = model_1, FUN = mySumm, nsim = 100, seed = 2047)


Bootstrap Statistics :
      original        bias    std. error
t1* 0.08195094  4.984615e-05 0.001343312
t2* 0.07022657 -3.549436e-04 0.005372882